# What happens:
Files are created, send email with excel script

## Import 

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import datetime
import win32com.client as win32
import calendar
from openpyxl import Workbook, load_workbook
from IPython.lib.display import Audio
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.cell import Cell
from openpyxl.styles import Color, Font, PatternFill

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [2]:
directory = "C:/Users/WZHARBC/Documents/Python Monthly Tasks/Exports/Recalibration of DQI"
os.chdir(directory)

rd = str(input("Please enter the RD (YYYYMMDD): "))

ws = "0002"

Please enter the RD (YYYYMMDD): 20221031


### Make connection with Oracle DB

In [3]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = ws))

## Function

In [4]:
def save_file(file_name, sheet_name, ws_nr, sqls):
    # Create file
    wb = Workbook()
    for index, sheet in enumerate(sheet_name):
        wb.create_sheet(sheet)
        
    std = wb.get_sheet_by_name('Sheet')
    wb.remove_sheet(std)
    wb.save(filename = file_name)
    
    dfs = []
    # Select SQLs
    for index, sheet in enumerate(sheet_name):
        try:
            connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'), {partition})".format(
                            reporting_date = rd, partition = ws_nr[index]))
            df_temp= pd.read_sql_query( sqls[index], connection_RAY)
            dfs.append(df_temp)
        except:
            dfs.append(pd.DataFrame())
            print( str(ws_nr[index]) + " is not created")
            
    dfs[0].to_csv(file_name, sep = ';', index = False,)
    print('DataFrame is written to Excel File successfully.')


### Data for Boris Peulic

In [5]:
file_name2 ='Retail Data ' + rd + '.csv'
sheet_name = ["Data" ]
ws_nr = [2]
sqls = ["""
with t_cdr_sub as (
    select exposure_attribute_5, min(npe_date) npe_date  
    from t_cdr 
   where booking_company in ('RBIPL_B')
        and counterparty_type_original IN ('1', '20')
        and npe_date is not null
    group by exposure_attribute_5
)
select 
    t.bs_type,
    t.default_flag,
    t.asset_class_original,
    t.attribute_5  ACCOUNT_ID ,
    t.counterparty_code  CUSTOMER_ID,
    substr(t.partition_key, 1, 8) PERIOD,
    t.BIS_ENTITY_TYPE_ORIGINAL  BUSINESS_SEG,
    t.counterparty_type  ASSET_CLASS,
    t.contract_type  GPC_CODE,
    t.start_date  OPEN_DATE,
    t.maturity_date  MATURITY_DATE,
    tcdr.npe_date LAST_DEFAULT_DATE,
    wrfb.fb_start_date FORB_START_DATE,
    wrfb.fb_start_date EBA_START_DATE,
    (select incorporation_country from entity te where t.counterparty_code = te.entity_code) CUSTOMER_COUNTRY,
    t.risk_region  CUSTOMER_REGION,
    t.currency  CURRENCY,
    wrfb.dpd_asof_repdate DPD,
    t.nace_code  NACE,
    wrfb.concession_type CONCESSION_TYPE,
    wrfb.exposure_classification EBA_STATUS,
        t.method_original  RWA_APPROACH,
--        '#n/a#'  COREP_SHEET,
    max(t.provision_type)  IFRS9_STAGE,
    --    '#n/a#'  IR_TYPE,
    sum(t.EAD_PRE_CCF) EAD_PRE_CCF,-- - sum on attribute_5 level,  BALANCE,
    --    '#n/a#'  OFF_BAL,
    0 LIMIT,
    sum(t.provisions)  IFRS9_PROVISIONS,
    sum(t.accruals)  ACCRUED_INTERESTS,
    sum(t.rwa)  RWA,
    AVG(t.ccf)  CF_STD,
    AVG(to_number(EIR, '999999999.99999999999999')) EIR
from rzb_cdr t
    left join rzb_forbearance wrfb on wrfb.booking_company in ('RBIPL') AND t.attribute_5 = wrfb.attribute_5
    left join t_cdr_sub tcdr on tcdr.exposure_attribute_5 = t.attribute_5
where 
    t.booking_company in ('RBIPL')
    and counterparty_type IN ('1', '20')
    and record_source = 'T_CDR'
group by
    t.bs_type,
    t.default_flag,
    t.asset_class_original,
    default_start_date,
    t.attribute_5, 
    t.counterparty_code , 
    substr(t.partition_key, 1, 8), 
    t.BIS_ENTITY_TYPE_ORIGINAL, 
    t.counterparty_type, 
    t.contract_type, 
    t.start_date, 
    t.maturity_date,
    tcdr.npe_date,
    wrfb.fb_start_date,
    wrfb.fb_start_date,
    t.risk_region, 
    t.currency,
    wrfb.dpd_asof_repdate,
    t.nace_code,
    wrfb.concession_type,
    wrfb.exposure_classification,
    t.method_original 
order by t.attribute_5
"""]
save_file(file_name2, sheet_name, ws_nr, sqls)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  


DataFrame is written to Excel File successfully.


### Emailing Retail Data to Boris Peulic

In [6]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'boris.peulic@rbinternational.com'
mail.Cc = 'markus.hochleitner@rbinternational.com;'
mail.Subject = 'Retail Data'
mail.Body = '''Dear Boris,

Please find attached the Retail data as of ''' + rd + '''.

Thanks in advance and kind regards,
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''
# To attach a file to the email (optional):
attachment  = directory + "/" + file_name2
mail.Attachments.Add(attachment)
mail.Display()
